In [1]:
from utils.BackTest import Engine, Strategy
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/features_trading_strategy.csv")

In [3]:
df = df.rename(columns={'OPEN':'Open','HIGH':'High','LOW':'Low','CLOSE':'Close','VOLUME':'Volume'})

In [4]:
df['Close']

0       51509.99
1       51995.58
2       51656.96
3       51456.01
4       51288.42
          ...   
8995    89278.88
8996    90035.37
8997    90154.00
8998    90300.01
8999    87132.80
Name: Close, Length: 9000, dtype: float64

In [5]:
class SMACrossOver(Strategy):
    def __init__(self):
        super().__init__()
        # Default multipliers for TP and SL calculations
        self.tp_atr_multiplier = 1.5   
        self.sl_atr_multiplier = 1.5  
    def on_bar(self):
        # Get current bar data
        current_bar = self.data.loc[self.current_idx]
        close = current_bar['Close']
        sma20 = current_bar['SMA20']
        sma50 = current_bar['SMA50']

        # For multiple positions, get all BTC positions
        btc_positions = [p for p in self.positions if p.ticker == 'BTC']

        # Buy signal: SMA20 > SMA50, and we allow up to 5 positions.
        if sma20 > sma50:
            if len(btc_positions) < 5:
                # Use 10% of available cash to determine the order size.
                order_size = 0.1 * self.cash / close
                self.buy('BTC', order_size)
                print(f"{self.current_idx} BUY signal: entering new position, order size {order_size:.4f}")
            else:
                print(f"{self.current_idx} BUY signal: maximum BTC positions reached.")


In [6]:
df['SMA50'] = df['Close'].rolling(window=50).mean()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            9000 non-null   object 
 1   Open                9000 non-null   float64
 2   High                9000 non-null   float64
 3   Low                 9000 non-null   float64
 4   Close               9000 non-null   float64
 5   TOTAL_TRADES        9000 non-null   int64  
 6   TOTAL_TRADES_BUY    9000 non-null   int64  
 7   TOTAL_TRADES_SELL   9000 non-null   int64  
 8   Volume              9000 non-null   float64
 9   VOLUME_BUY          9000 non-null   float64
 10  VOLUME_SELL         9000 non-null   float64
 11  return              8999 non-null   float64
 12  log_return          8999 non-null   float64
 13  hourly_volatility   8988 non-null   float64
 14  EMA_8               9000 non-null   float64
 15  EMA_13              9000 non-null   float64
 16  EMA_21

In [8]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')

In [9]:
df_filtered = df.dropna()
df_filtered

,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,Volume,VOLUME_BUY,VOLUME_SELL,...,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR,ATR_Signal,SMA50
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-02 02:00:00+00:00,62030.01,62216.71,61949.03,62111.18,56088,24390,31698,1153.46042,607.80296,545.65746,...,1153.46042,-263.362181,1,267.68,186.70,80.98,267.68,384.756667,0,61999.6558
2024-03-02 03:00:00+00:00,62111.18,62426.70,62025.00,62307.45,67185,27772,39413,1465.47878,802.61870,662.86008,...,1465.47878,-98.710661,0,401.70,315.52,86.18,401.70,347.451667,0,62018.2834
2024-03-02 04:00:00+00:00,62307.45,62307.45,62035.95,62035.95,56098,24794,31304,1147.71004,516.23228,631.47776,...,-1147.71004,-198.615364,-1,271.50,0.00,271.50,271.50,333.268333,0,62024.1966
2024-03-02 05:00:00+00:00,62035.95,62079.99,61752.30,62006.55,65329,23953,41376,1415.65823,637.30577,778.35246,...,-1415.65823,-314.524208,0,327.69,44.04,283.65,327.69,326.113333,0,62033.1010
2024-03-02 06:00:00+00:00,62006.55,62256.00,61930.00,61946.00,57826,25109,32717,1059.53378,543.92419,515.60959,...,-1059.53378,-385.477501,0,326.00,249.45,76.55,326.00,325.211667,0,62023.1430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03 14:00:00+00:00,93586.83,93721.37,89136.45,89278.88,909864,449798,460066,6172.59027,2600.65274,3571.93753,...,-6172.59027,91.667872,-1,4584.92,134.54,4450.38,4584.92,1402.496667,-1,89026.0262
2025-03-03 15:00:00+00:00,89278.88,91307.00,89117.00,90035.37,815439,415993,399446,5771.66863,2921.20810,2850.46053,...,5771.66863,632.620325,1,2190.00,2028.12,161.88,2190.00,1618.750000,0,89136.0888
2025-03-03 16:00:00+00:00,90035.37,90750.91,89691.02,90154.00,448660,224630,224030,2111.23659,1038.58417,1072.65242,...,2111.23659,773.440922,0,1059.89,715.54,344.35,1059.89,1622.873333,0,89244.8890


In [10]:
e = Engine()
e.add_data(df_filtered)
e.add_strategy(SMACrossOver())
stats = e.run()     

  2%|▏         | 156/8801 [00:00<00:05, 1552.28it/s]

2024-03-02 02:00:00+00:00 BUY signal: entering new position, order size 0.1610
2024-03-02 03:00:00+00:00 New position opened: <Position: BTC size: 0.1610016103381066 entry: 62111.18>
2024-03-02 03:00:00+00:00 BUY signal: entering new position, order size 0.1605
2024-03-02 04:00:00+00:00 New position opened: <Position: BTC size: 0.16049445130558224 entry: 62307.45>
2024-03-02 04:00:00+00:00 BUY signal: entering new position, order size 0.1612
2024-03-02 05:00:00+00:00 Stop Loss Filled. Limit: 61807.5475 / Low: 61752.3
2024-03-02 05:00:00+00:00 Position closed for ticker BTC at 61807.5475
2024-03-02 05:00:00+00:00 New position opened: <Position: BTC size: 0.16119685440458315 entry: 62035.95>
2024-03-02 05:00:00+00:00 BUY signal: entering new position, order size 0.1613
2024-03-02 06:00:00+00:00 New position opened: <Position: BTC size: 0.16127328483845657 entry: 62006.55>
2024-03-02 06:00:00+00:00 BUY signal: entering new position, order size 0.1614
2024-03-02 07:00:00+00:00 New position

  4%|▎         | 312/8801 [00:00<00:05, 1498.69it/s]

2024-03-11 16:00:00+00:00 New position opened: <Position: BTC size: 0.13886960144424385 entry: 72010.0>
2024-03-11 16:00:00+00:00 BUY signal: maximum BTC positions reached.
2024-03-11 17:00:00+00:00 BUY signal: maximum BTC positions reached.
2024-03-11 18:00:00+00:00 BUY signal: maximum BTC positions reached.
2024-03-11 19:00:00+00:00 Take Profit Filled. Limit: 72791.76000000001 / High: 72800.0
2024-03-11 19:00:00+00:00 Position closed for ticker BTC at 72791.76000000001
2024-03-11 19:00:00+00:00 Take Profit Filled. Limit: 72525.76999999999 / High: 72800.0
2024-03-11 19:00:00+00:00 Position closed for ticker BTC at 72525.76999999999
2024-03-11 19:00:00+00:00 Take Profit Filled. Limit: 72508.95999999999 / High: 72800.0
2024-03-11 19:00:00+00:00 Position closed for ticker BTC at 72508.95999999999
2024-03-11 19:00:00+00:00 BUY signal: entering new position, order size 0.1389
2024-03-11 20:00:00+00:00 New position opened: <Position: BTC size: 0.1389467833819647 entry: 71970.0>
2024-03-11 2

  7%|▋         | 595/8801 [00:00<00:03, 2092.21it/s]

...

2025-02-07 21:00:00+00:00 New position opened: <Position: BTC size: 0.10442303609768608 entry: 95764.31>
2025-02-07 21:00:00+00:00 BUY signal: entering new position, order size 0.1042
2025-02-07 22:00:00+00:00 Stop Loss Filled. Limit: 95989.97499999999 / Low: 95632.23
2025-02-07 22:00:00+00:00 Position closed for ticker BTC at 95989.97499999999
2025-02-07 22:00:00+00:00 New position opened: <Position: BTC size: 0.10415427664750715 entry: 96011.42>
2025-02-08 02:00:00+00:00 Take Profit Filled. Limit: 96723.13500000001 / High: 96818.33
2025-02-08 02:00:00+00:00 Position closed for ticker BTC at 96723.13500000001
2025-02-08 03:00:00+00:00 Take Profit Filled. Limit: 96744.1075 / High: 96880.0
2025-02-08 03:00:00+00:00 Position closed for ticker BTC at 96744.1075
2025-02-08 05:00:00+00:00 Stop Loss Filled. Limit: 96199.48499999999 / Low: 96064.61
2025-02-08 05:00:00+00:00 Position closed for ticker BTC at 96199.48499999999
2025-02-09 13:00:00+00:00 BUY signal: entering new position, order s

 97%|█████████▋| 8517/8801 [00:03<00:00, 2188.68it/s]

2025-02-15 09:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-02-15 10:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-02-15 11:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-02-15 12:00:00+00:00 Take Profit Filled. Limit: 97777.73249999998 / High: 97833.42
2025-02-15 12:00:00+00:00 Position closed for ticker BTC at 97777.73249999998
2025-02-15 12:00:00+00:00 Take Profit Filled. Limit: 97809.55249999999 / High: 97833.42
2025-02-15 12:00:00+00:00 Position closed for ticker BTC at 97809.55249999999
2025-02-15 12:00:00+00:00 BUY signal: entering new position, order size 0.1023
2025-02-15 13:00:00+00:00 Take Profit Filled. Limit: 97867.7925 / High: 97962.51
2025-02-15 13:00:00+00:00 Position closed for ticker BTC at 97867.7925
2025-02-15 13:00:00+00:00 New position opened: <Position: BTC size: 0.10229779255685352 entry: 97753.82>
2025-02-15 13:00:00+00:00 BUY signal: entering new position, order size 0.1023
2025-02-15 14:00:00+00:00 New position opened:

100%|██████████| 8801/8801 [00:03<00:00, 2222.87it/s]


2025-03-02 04:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 05:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 06:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 07:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 08:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 09:00:00+00:00 Stop Loss Filled. Limit: 85686.99749999998 / Low: 85633.13
2025-03-02 09:00:00+00:00 Position closed for ticker BTC at 85686.99749999998
2025-03-02 09:00:00+00:00 BUY signal: entering new position, order size 0.1164
2025-03-02 10:00:00+00:00 New position opened: <Position: BTC size: 0.11641267361495111 entry: 85901.3>
2025-03-02 10:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 11:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 12:00:00+00:00 BUY signal: maximum BTC positions reached.
2025-03-02 13:00:00+00:00 Stop Loss Filled. Limit: 85538.2225 / Low: 85477.81
2025-03-02 13:00:00+00:0

In [11]:
e.plot()